In [1]:
import xarray as xr
import numpy as np
import earthkit.data
from earthkit.transforms import aggregate
from dask.distributed import LocalCluster

In [2]:
cluster = LocalCluster()
client = cluster.get_client()
client

/Users/mastermaps/miniforge3/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 62523 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:62523/status,
Dashboard: http://127.0.0.1:62523/status,Workers: 4
Total threads: 12,Total memory: 96.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:62524,Workers: 4
Dashboard: http://127.0.0.1:62523/status,Total threads: 12
Started: Just now,Total memory: 96.00 GiB
Comm: tcp://127.0.0.1:62539,Total threads: 3
Dashboard: http://127.0.0.1:62542/status,Memory: 24.00 GiB
Nanny: tcp://127.0.0.1:62527,


In [10]:
pm_file = "./data/pm_final_srilanka_linearp.nc"
pm_ds = xr.open_dataset(pm_file, chunks={"time": 100})
pm_data = pm_ds["__xarray_dataarray_variable__"]

pop_file = "./data/lka_pop_2023_CN_1km_R2025A_UA_v1.tif"
pop_ds = xr.open_dataset(pop_file)
pop_data = pop_ds['band_data'].rename({"x": "lon", "y": "lat"})
pop_aligned = pop_data.interp(lon=pm_data.lon, lat=pm_data.lat, method="nearest")

pm_weighted = pm_data * pop_aligned

district_file = "./data/geoBoundaries-LKA-ADM2.geojson"
features = earthkit.data.from_source("file", district_file)

agg_num = aggregate.spatial.reduce(pm_weighted, features, how="sum", mask_dim="shapeName")
agg_den = aggregate.spatial.reduce(pop_aligned, features, how="sum", mask_dim="shapeName")

pw = agg_num / agg_den
pw.name = "pm25_popweighted"

agg_df = pw.to_dataframe().reset_index()

agg_df

,shapeName,time,band,spatial_ref,pm25_popweighted
0,Jaffna District,2020-03-01,1,0,43.995122
1,Jaffna District,2020-03-02,1,0,64.471165
2,Jaffna District,2020-03-03,1,0,83.498082
3,Jaffna District,2020-03-04,1,0,50.150797
4,Jaffna District,2020-03-05,1,0,38.137115
...,...,...,...,...,...
35020,Trincomalee District,2023-12-27,1,0,18.800796
35021,Trincomalee District,2023-12-28,1,0,22.124807
35022,Trincomalee District,2023-12-29,1,0,18.672990
35023,Trincomalee District,2023-12-30,1,0,26.025079


In [9]:
pm_file = "./data/pm_final_srilanka_linearp.nc"
pm_ds = xr.open_dataset(pm_file, chunks={"time": 100})
pm_data = pm_ds["__xarray_dataarray_variable__"]

district_file = "./data/geoBoundaries-LKA-ADM2.geojson"
features = earthkit.data.from_source("file", district_file)

years = np.unique(pm_data['time'].dt.year).astype(str)

for year in years:
  print(f"Calculating {year}") 
  pm_year = pm_data.sel(time=year) 
    
  pop_file = f"./data/lka_pop_{year}_CN_1km_R2025A_UA_v1.tif"
  pop_ds = xr.open_dataset(pop_file)
  pop_data = pop_ds['band_data'].rename({"x": "lon", "y": "lat"})
  pop_aligned = pop_data.interp(lon=pm_year.lon, lat=pm_year.lat, method="nearest")

  pm_weighted = pm_year * pop_aligned

  agg_num = aggregate.spatial.reduce(pm_weighted, features, how="sum", mask_dim="shapeName")
  agg_den = aggregate.spatial.reduce(pop_aligned, features, how="sum", mask_dim="shapeName")

  pw = agg_num / agg_den
  pw.name = "pm25_popweighted"

  agg_df = pw.to_dataframe().reset_index()

Calculating 2020
Calculating 2021
Calculating 2022
Calculating 2023
